In [17]:
import numpy as np
from numpy.polynomial.chebyshev import Chebyshev
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.special import eval_chebyt

In [19]:
def __Levi_integral(dw, h, m, num, seed=None):
    """Интеграл Леви для вычисления аппроксимации интеграла Ито
    num --- число членов ряда"""
    if seed != None:
        np.random.seed(seed)
        
    X = np.random.normal(loc=0.0, scale=1.0, size=(m, num))
    Y = np.random.normal(loc=0.0, scale=1.0, size=(m, num))

    K = range(0, num, 1)

    A1 = [np.outer(X[:, k], (Y[:, k] + np.sqrt(2.0/h)*dw)) for k in K]
    A2 = [np.outer((Y[:, k] + np.sqrt(2.0/h)*dw), X[:, k]) for k in K]
    A = ((1.0/(k+1))*(A1[k] - A2[k]) for k in K)
    A = (h/np.pi)*np.sum(A)

    return A


def Ito1Wm(dW, h):
    """Генерирование значений однократного интеграла Ито для многомерного
    винеровского процесса"""
    (N, m) = dW.shape
    Ito = np.empty(shape=(N, m+1))
    Ito[:, 0] = h
    Ito[:, 1:] = dW[:, :]
    return Ito


def Ito2Wm(dW, h, n, seed=None):
    """Вычисление интеграла Ито (по точной формуле или по аппроксимации)
    Возвращает список матриц I(i,j)"""
    (N, m) = dW.shape
    E = np.identity(m)
    Ito = np.empty(shape=(N, m+1, m+1))
    if seed != None:
        np.random.seed(seed)
    dzeta = np.random.normal(loc=0, scale=h, size=(N, m))
    Ito[:, 0, 0] = h
    for dw, I, dz in zip(dW, Ito, dzeta):
        I[0, 1:] = 0.5*h*(dw - dz / np.sqrt(3))
        I[1:, 0] = 0.5*h*(dw + dz / np.sqrt(3))
        I[1:, 1:] = 0.5*(np.outer(dw, dw) - h*E) + __Levi_integral(dw, h, m, n, seed+1)
    return Ito

In [21]:
def chebyt_derivative(n, x, eps=1e-6):
    """Численная производная T_n(x) (можно заменить аналитической формулой)"""
    return (eval_chebyt(n, x + eps) - eval_chebyt(n, x - eps)) / (2 * eps)

In [23]:
seed = 42
s = 3
m = 4
eta = 4.3
T = 1
tau = 0.25
N_true = 2**12 

In [25]:
B = np.array([[0, 0, 0], [1, 0, 0], [-1, 0, 0]])  # Параметры B
beta1 = np.array([1, 0, 0])  # Параметры β1
beta2 = np.array([0, 0.5, -0.5])  # Параметры β2
alpha = np.array([1/3, 1/3, 1/3])
A = np.tile(alpha, (s, 1))

In [27]:
def f(y):
    return -2.0*y  # Детерминированная часть

def g(y, yd):
    return [0.1*y + 0.2*yd for _ in range(m)] # Стохастическая часть с запаздыванием
    
def y0(t):
    return np.ones_like(t)  # История: y(t) = 1 при t ∈ [-τ, 0]

In [31]:
def srockd1_sdde(f, g, y0, T, tau, M, dW, I1, I2, h, N, m, s, eta, beta1, beta2, B, A, seed):         
    y = np.zeros(N+1)
    t_history = np.linspace(-tau, 0, M+1)
    y[:M+1] = y0(t_history)
    t = np.linspace(0, T, N + 1)
    for n in range(N):
        
        y_curr = y[n]
        y_prev = y[n - M] if n-M >= 0 else y0(t[n] - tau)
        
        K = np.zeros(s+1)
        omega0 = 1 + eta/ s**2
        Ts_omega0 = eval_chebyt(s, omega0)
        Ts_prime_omega0 = chebyt_derivative(s, omega0)  # Производная T_s(omega0)
        omega1 = Ts_omega0 / Ts_prime_omega0
        K[0] = y_curr
        K[1] = y_curr + h * (omega1 / omega0) * f(K[0])
        for j in range(2, s+1):
            T_j = eval_chebyt(j, omega0)
            T_jm1 = eval_chebyt(j-1, omega0)
            T_jm2 = eval_chebyt(j-2, omega0)
            K[j] = (2 * T_jm1 / T_j) * (h * omega1 * f(K[j-1])) + (2 * omega0 * T_jm1 / T_j) * K[j-1] - (T_jm2 / T_j) * K[j-2]
        H = np.zeros((s, m))
        for v in range(m):  
            for i in range(s): 
                # Детерминированная часть
                det_part = y_curr + sum(A[i, k-1] * h * f(K[k-1]) for k in range(1,s+1))
                # Стохастическая часть 
                stoch_part = 0.0
                for k in range(i):  # Сумма по i_b (k < i)
                    for l in range(m):
                        gl = g(H[i,l], y_prev)[l]
                        stoch_part += B[i, k] * gl * I2[n][l+1][v+1]  # I2[l][j] = ζ̃^(l,j)
                H[i][v] = det_part + stoch_part
        # Обновление решения
        y[n+1] = K[s]
        stoch_term = 0
        for i in range(s):
            for v in range(m):
                gv = g(H[i, v], y_prev)[v]
                stoch_term += beta1[i] * gv * I1[n, v+1]  # ΔW_j
                stoch_term += beta2[i] * gv * np.sqrt(h)  # √h·ζ 
        y[n+1] += stoch_term
        
    return y[-1]

In [35]:
def sri1_sdde(f, g, y0, tau, dW, T, N, h, I1, I2, m):
    M = int(np.ceil(tau / h))
    y = np.zeros(N+1)
    t_history = np.linspace(-tau, 0, M+1)
    y[:M+1] = y0(t_history)
    t = np.linspace(0, T, N + 1)
    
    for n in range(N):
        y_curr = y[n]
        delay_idx = n - M
        y_delay = y[delay_idx] if delay_idx >= 0 else y0(t[n] - tau)
        H2 = np.zeros(m)
        H3 = np.zeros(m)
        for k in range(m):
            H2[k] = y_curr
            H3[k] = y_curr
            for l in range(m):
                gl = g(y_curr,y_delay)[l]
                H2[k] += gl * I2[n][l+1][k+1]/np.sqrt(h)
                H3[k] -= gl * I2[n][l+1][k+1]/np.sqrt(h)
        y[n+1] = y_curr + f(y_curr)*h
        for j in range(m):
            gj1 = g(y_curr, y_delay)[j]
            gj2 = g(H2[j], y_delay)[j]
            gj3 = g(H3[j], y_delay)[j]
            y[n+1] += gj1 * I1[n,j+1] + 1/2*(gj2-gj2)*np.sqrt(h)
    return y[-1]

In [39]:
num_exp = 1000
setH = 5

In [ ]:
dt = T/N_true
strongerr = np.zeros((setH,num_exp))
weakerr = np.zeros(setH)
line = np.zeros(setH)
for d in range(num_exp):[
    print(d)
    dW = np.sqrt(dt) * np.random.normal(size=(N_true, m))
    I1_true = Ito1Wm(dW, dt)
    I2_true = Ito2Wm(dW, dt, 10, seed = 42)
    Ytrue = sri1_sdde(f,g, y0, tau, dW, T, N_true, dt, I1_true, I2_true, m)
    for p in range(setH):
        R = 2**(setH+p-1)
        N_temp = int(N_true/R)
        DT = T/N_temp
        M = int(tau / DT)
        line[p] = dt*R
        dWinc = np.zeros((N_temp,m))
        for n in range(N_temp):
            for j in range(m):
                dWinc[n, j] = np.sum(dW[n*R : (n+1)*R, j])
        I1 = Ito1Wm(dWinc, DT)
        I2 = Ito2Wm(dWinc, DT, 10, seed = 42)
        Yrock = srockd1_sdde(f, g, y0, tau, T, M, dWinc, I1, I2, DT, N_temp, m, s, eta, beta1, beta2, B, A, seed)
        strongerr[p,d] = np.abs(Ytrue-Yrock)

In [ ]:
Strong = np.mean(strongerr, axis=1)

In [ ]:
plt.figure(figsize=(12, 5))

# График сильной сходимости
plt.subplot(1, 2, 2)
plt.loglog(line, Strong, '*-', label='Сильная ошибка для S-ROCK1')
plt.loglog(line, (line**(1) / np.sqrt(line[0])) * Strong[0], '--', label='Наклон 1')
plt.ylabel("$E[|Y_N-Y(T)|]$", fontsize=12)
plt.title("Сильная сходимость", fontsize=14)
plt.xlabel("h", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.show()

In [ ]:
weak_errors = np.zeros(setH)
for p in range(setH):
    R = 2**(setH+p-1)
    N_temp = int(N_true/R)
    DT = T/N_temp
    M = int(tau / DT)
    Ytrue_weak = np.zeros(num_exp)
    Yrock_weak = np.zeros(num_exp)
    for d in range(num_exp):
        print(d)
        dW = np.sqrt(dt) * np.random.normal(size=(N_true, m))
        I1_true = Ito1Wm(dW, dt)
        I2_true = Ito2Wm(dW, dt, 100, seed = 42)
        dWinc = np.zeros((N_temp, m))
        
        for n in range(N_temp):
            for j in range(m):
                dWinc[n, j] = np.sum(dW[n*R : (n+1)*R, j])
        I1 = Ito1Wm(dWinc, DT)
        I2 = Ito2Wm(dWinc, DT, 100, seed = 42)
        Ytrue_weak[d] = sri1_sdde(f, g, y0, tau, dW, T, N_true, dt, I1_true, I2_true, m)
        Yrock_weak[d] = srockd1_sdde(f, g, y0, tau, T, M, dWinc, I1, I2, DT, N_temp, m, s, eta, beta1, beta2, B, A, seed)
    weak_errors[p] = np.abs(np.mean(Ytrue_weak) - np.mean(Yrock_weak))

In [ ]:
plt.figure(figsize=(12, 5))

# График слабой сходимости
plt.subplot(1, 2, 2)
plt.loglog(line, weak_errors, '*-', label='Слабая ошибка для S-ROCK1')
plt.loglog(line, (line**(1) / np.sqrt(line[0])) * weak_errors[0], '--', label='Наклон 1')
plt.title("Слабая сходимость", fontsize=14)
plt.xlabel("h", fontsize=12)
plt.ylabel("$|E[Y_n] - E[Y(T)]|$", fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.show()